In [1]:
import pandas as pd

Limpieza de los datos

In [2]:
df_historico = pd.read_csv('datos/fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('datos/fifa_worldcup_fixture.csv')
df_faltante = pd.read_csv('datos/fifa_worldcup_missing_data.csv')

In [3]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

Limpieza de la data faltante y posteriormente agregarla al df de data histórica

In [4]:
#eliminamos nulos
df_faltante.dropna(inplace = True)

In [5]:
#acoplamos los dataframes
df_historico_completo = pd.concat([df_historico, df_faltante], ignore_index = True)

In [6]:
#eliminamos duplicados
df_historico_completo.drop_duplicates(inplace = True)
#ordenamos el dataframe por el año
df_historico_completo.sort_values('year', inplace = True)

Limpieza del dataframe completo

In [7]:
#eliminamos datos no numericos del fixture
index_partido = df_historico_completo[df_historico_completo['home'].str.contains('Sweden') &
                                      df_historico_completo['away'].str.contains('Austria')].index

df_historico_completo.drop(index = index_partido, inplace = True)

In [8]:
#eliminamos datos que no sean una barra o un numero decimal dentro de la columna score
df_historico_completo['score'] = df_historico_completo['score'].str.replace('[^\d–]', '', regex = True)

In [9]:
#verificamos el procedimiento anterior
df_historico_completo[df_historico_completo['score'].str.contains('[^\d–]')]

,home,score,away,year


In [10]:
#quitamos espacios vacios del dataframe completo
df_historico_completo['home'] = df_historico_completo['home'].str.strip()
df_historico_completo['away'] = df_historico_completo['away'].str.strip()

In [11]:
#separamos la columna score en dos columnas distintas
df_historico_completo[['goles_local','goles_visitante']] = df_historico_completo['score'].str.split('–', expand = True)
#dropeamos la columna score
df_historico_completo.drop('score', axis = 1, inplace = True)

In [15]:
#renombramos las columnas
df_historico_completo.rename(columns = {'home': 'Local', 'away': 'Visitante',
                                         'year': 'Año'} , inplace = True)

In [23]:
#cambiamos los tipos de columnas necesarias
df_historico_completo = df_historico_completo.astype({'goles_local': int,
                                                       'goles_visitante': int})

In [25]:
#creamos una columna con informacion relevante
df_historico_completo['goles_por_partido'] = df_historico_completo['goles_local'] + df_historico_completo['goles_visitante']

Por ultimo, exportamos los dataframes pasados por el proceso de limpieza y normalizacion

In [27]:
df_historico_completo.to_csv('clean_fifa_worldcup_matches.csv' ,index = False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv' ,index = False)